In [1]:
import numpy as np
import pandas as pd
import string
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

df = pd.read_csv('emails.csv')
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

stop = stopwords.words('english')
df['text'] = df['text'].map(lambda text:re.sub('[^a-zA-Z-\']{1,}', ' ',text))
df.text = [x.strip('Subject') for x in df.text]
df['text'] = df['text'].apply(lambda x: x.split())
df['text']=df['text'].apply(lambda text_list: list(filter(lambda text:text not in stop,text_list)))

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
ltr = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FALCON\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FALCON\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df['text']=df['text'].apply(lambda text_list: list(map(lambda word:ltr.lemmatize(word,'n'),text_list)))
df['text']=df['text'].apply(lambda text_list: list(map(lambda word:ltr.lemmatize(word,'v'),text_list)))
df['text']=df['text'].apply(lambda text_list: list(map(lambda word:ltr.lemmatize(word,'a'),text_list)))
df['text']=df['text'].apply(lambda text_list: list(map(lambda word:ltr.lemmatize(word,'r'),text_list)))

df["text"] = df['text'].apply(lambda rem : (list(filter(lambda x : len(x) > 2 ,rem))))

In [3]:
corpus = df['text'].apply(' '.join)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=10)
X = cv.fit_transform(corpus).toarray()
Y = df.iloc[:, 1].values

x=pd.DataFrame(X)
vocab_size = x.shape[1]
d = {'Spam': Y}
y=pd.DataFrame(data = d)
full_matrix = pd.merge(x,y,left_index=True,right_index=True,how='inner')

In [5]:
#Splitting train and test data randomly in 70-30 ratio
split = np.random.rand(len(full_matrix)) < 0.3
test=full_matrix[split]
train=full_matrix[~split]


In [6]:
#Labeling X_train and Y_train for training data
Y_train=train.loc[:,'Spam']
X_train = train.loc[:,train.columns != 'Spam']

In [7]:
Total_mails = X_train.shape[0]
Total_mails

4072

In [8]:
Total_spam = Y_train.sum()
Total_spam

997

Calculating P(SPAM)


In [9]:
prob_of_spam = (Total_spam / Total_mails)
prob_of_spam

0.24484282907662083

In [10]:
words_per_emails = X_train.sum(axis=1)
#words_per_emails

In [11]:
total_words = words_per_emails.sum()
total_words

488575

In [12]:
Train_Spam_matrix = train.loc[train['Spam'] == 1]
Train_Spam_matrix.shape[0]


997

In [13]:
Train_Spam_matrix.drop('Spam',axis=1,inplace=True)
Total_words_in_spam_mails = (Train_Spam_matrix.sum(axis=1)).sum()
Total_words_in_spam_mails

C:\Users\FALCON\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


94698

Creating Non-Spam or Ham for Trainning data

In [14]:
Train_Ham_matrix = train.loc[train['Spam'] == 0]
Train_Ham_matrix.shape[0]


3075

In [15]:
Train_Ham_matrix.drop('Spam',axis=1,inplace=True)
Total_words_in_ham_mails = (Train_Ham_matrix.sum(axis=1)).sum()
Total_words_in_ham_mails

393877

In [16]:
words_in_spam = Train_Spam_matrix.sum(axis=0)+1
words_in_ham = Train_Ham_matrix.sum(axis=0)+1

Calculating P(Word|Spam)

In [17]:
prob_words_spam = words_in_spam / (Total_words_in_spam_mails + vocab_size)

Calculating P(Word|Ham)

In [18]:
prob_words_ham = words_in_ham / (Total_words_in_ham_mails + vocab_size)

P(Words) - probability that a word occurs

In [19]:
prob_words_all = (X_train.sum(axis=0)+1) / (total_words + vocab_size)

In [21]:
#Labeling X_test and Y_test for test data
Y_test=test.loc[:,'Spam']
X_test = test.loc[:,test.columns != 'Spam']

In [22]:
log_spam_words = X_test.dot(np.log(prob_words_spam) + np.log (prob_of_spam))
log_ham_words  = X_test.dot(np.log(prob_words_ham)  + np.log (1 - prob_of_spam))
prediction = log_spam_words > log_ham_words

In [23]:
Correct_pred = (Y_test == prediction).sum()
Wrong_pred = X_test.shape[0] - Correct_pred
print('Accuracy of Naive Bayes classifier: ', Correct_pred/X_test.shape[0])

Accuracy:  0.862600123228589
